In [2]:
import os
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv
import json


In [ ]:
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

MODEL='gpt-4o-mini'
openai = OpenAI()

In [4]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
ticket_prices = {"london":"$599","berlin":"$799","rejkjavik":"$5999","bengaluru":"$599","sfo":"$399"}

def get_ticket_price(destination_city):
    print(f"Get ticket price called for the city: {destination_city}")
    return ticket_prices.get(destination_city.lower(), "Unknown")

In [16]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [17]:
tools = [{"type": "function", "function": price_function}]

In [18]:
def chat(message, history):
    messages = [{"role":"system", "content":system_message}] + history + [{"role":"user","content":message}]
    response = openai.chat.completions.create(model=MODEL,messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)

        response = openai.chat.completions.create(model=MODEL, messages=messages)

    return response.choices[0].message.content

In [25]:
def handle_tool_call(message):
    print(f"Got a handle_tool_call: {message}")
    
    tool_call = message.tool_calls[0]
    print(f"Tool call object: {tool_call}")
    
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)

    response = {
        "role":"tool",
        "content" : json.dumps({"destination_city":city, "price":price}),
        "tool_call_id":tool_call.id        
    }
    return response, city

In [26]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.


Got a handle_tool_call: ChatCompletionMessage(content=None, refusal=None, role='assistant', audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_0S65GyYbu8ICOFDbvfu1b2GZ', function=Function(arguments='{"destination_city":"sfo"}', name='get_ticket_price'), type='function')])
Tool call object: ChatCompletionMessageToolCall(id='call_0S65GyYbu8ICOFDbvfu1b2GZ', function=Function(arguments='{"destination_city":"sfo"}', name='get_ticket_price'), type='function')
Get ticket price called for the city: sfo
